# Étape 1 : Exploration et Analyse des Données

## Objectifs
- Explorer les données enrichies
- Identifier les données personnelles présentes
- Préparer l'anonymisation avec spaCy NER
- Analyser les risques RGPD

In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
plt.style.use('default')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## 1. Chargement des Données

In [ ]:
# Charger les données enrichies
train_df = pd.read_csv('../data/raw/train_advanced.csv')
test_df = pd.read_csv('../data/raw/test_advanced.csv')

print(f"📊 Données d'entraînement: {train_df.shape}")
print(f"📊 Données de test: {test_df.shape}")

# Afficher les premières lignes
print("\n🔍 Aperçu des données d'entraînement:")
train_df.head()

In [ ]:
# Informations sur les colonnes
print("📋 Colonnes disponibles:")
for i, col in enumerate(train_df.columns):
    print(f"{i+1:2d}. {col}")

print("\n📈 Types de données:")
print(train_df.dtypes)

## 2. Analyse des Données Personnelles Ajoutées

In [ ]:
# Analyser les données personnelles ajoutées par le script d'enrichissement
personal_data_columns = ['username', 'email', 'ip_address', 'phone_number', 'country']

print("🔒 DONNÉES PERSONNELLES AJOUTÉES:")
print("=" * 50)

for col in personal_data_columns:
    if col in train_df.columns:
        print(f"\n📧 {col.upper()}:")
        print(f"   Exemples: {train_df[col].head(3).tolist()}")
        print(f"   Valeurs uniques: {train_df[col].nunique()}")

## 3. Analyse des Commentaires avec spaCy NER

In [ ]:
# Charger le modèle spaCy
print("🚀 Chargement du modèle spaCy...")
nlp = spacy.load('en_core_web_lg')
print("✅ Modèle chargé avec succès!")

# Test rapide
test_text = "Hello John Doe, please contact me at john.doe@email.com or call +1-555-123-4567"
doc = nlp(test_text)

print("\n🧪 Test de reconnaissance d'entités:")
print(f"Texte: {test_text}")
print("Entités détectées:")
for ent in doc.ents:
    print(f"  - {ent.text} ({ent.label_}) : {spacy.explain(ent.label_)}")

In [ ]:
# Analyser un échantillon de commentaires pour détecter des entités
def analyze_entities_in_comments(df, sample_size=100):
    """Analyse les entités nommées dans un échantillon de commentaires"""
    
    sample_comments = df['comment_text'].dropna().sample(min(sample_size, len(df))).tolist()
    
    all_entities = []
    entity_counts = Counter()
    
    print(f"🔍 Analyse de {len(sample_comments)} commentaires...")
    
    for i, comment in enumerate(sample_comments):
        if i % 20 == 0:
            print(f"  Progression: {i}/{len(sample_comments)}")
            
        # Limiter la longueur pour éviter les timeouts
        if len(comment) > 1000:
            comment = comment[:1000]
            
        try:
            doc = nlp(comment)
            for ent in doc.ents:
                all_entities.append({
                    'text': ent.text,
                    'label': ent.label_,
                    'description': spacy.explain(ent.label_),
                    'comment_id': i
                })
                entity_counts[ent.label_] += 1
        except Exception as e:
            print(f"Erreur avec commentaire {i}: {str(e)[:50]}")
            continue
    
    return all_entities, entity_counts

# Lancer l'analyse
entities, entity_counts = analyze_entities_in_comments(train_df, sample_size=50)

print("\n📊 RÉSULTATS DE L'ANALYSE NER:")
print("=" * 40)
print(f"Total d'entités trouvées: {len(entities)}")
print("\nTypes d'entités les plus fréquents:")
for label, count in entity_counts.most_common(10):
    print(f"  {label:10s}: {count:3d} ({spacy.explain(label)})")

In [ ]:
# Afficher quelques exemples d'entités détectées
if entities:
    entities_df = pd.DataFrame(entities)
    
    print("\n🔍 EXEMPLES D'ENTITÉS DÉTECTÉES:")
    print("=" * 50)
    
    # Grouper par type d'entité
    for label in ['PERSON', 'ORG', 'GPE', 'EMAIL', 'PHONE']:
        if label in entities_df['label'].values:
            examples = entities_df[entities_df['label'] == label]['text'].unique()[:5]
            print(f"\n{label} ({spacy.explain(label)}):")
            for example in examples:
                print(f"  - {example}")
else:
    print("⚠️ Aucune entité détectée dans l'échantillon")

## 4. Détection de Patterns de Données Personnelles

In [ ]:
# Patterns regex pour détecter des données personnelles
patterns = {
    'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
    'phone': r'\b(?:\+?1[-.]?)?(?:\(?\d{3}\)?[-.]?)?\d{3}[-.]?\d{4}\b',
    'ip_address': r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b',
    'credit_card': r'\b(?:\d{4}[-\s]?){3}\d{4}\b',
    'ssn': r'\b\d{3}-\d{2}-\d{4}\b'
}

def find_personal_data_patterns(text_series, max_samples=1000):
    """Trouve des patterns de données personnelles dans les textes"""
    
    results = {}
    
    # Prendre un échantillon pour l'analyse
    sample = text_series.dropna().sample(min(max_samples, len(text_series)))
    
    print(f"🔍 Recherche de patterns dans {len(sample)} commentaires...")
    
    for pattern_name, pattern in patterns.items():
        matches = []
        for text in sample:
            if isinstance(text, str):
                found = re.findall(pattern, text, re.IGNORECASE)
                matches.extend(found)
        
        results[pattern_name] = {
            'count': len(matches),
            'examples': list(set(matches))[:5] if matches else []
        }
    
    return results

# Analyser les commentaires
personal_data_found = find_personal_data_patterns(train_df['comment_text'])

print("\n🚨 DONNÉES PERSONNELLES DÉTECTÉES DANS LES COMMENTAIRES:")
print("=" * 60)

for data_type, info in personal_data_found.items():
    print(f"\n{data_type.upper()}:")
    print(f"  Occurrences trouvées: {info['count']}")
    if info['examples']:
        print(f"  Exemples: {info['examples']}")
    else:
        print(f"  Aucun exemple trouvé")

## 5. Statistiques Générales sur les Données

In [ ]:
# Statistiques de base
print("📊 STATISTIQUES GÉNÉRALES:")
print("=" * 30)

print(f"\n📈 Taille des datasets:")
print(f"  Train: {train_df.shape[0]:,} lignes, {train_df.shape[1]} colonnes")
print(f"  Test:  {test_df.shape[0]:,} lignes, {test_df.shape[1]} colonnes")

print(f"\n📝 Longueur des commentaires (train):")
train_df['comment_length'] = train_df['comment_text'].str.len()
print(f"  Moyenne: {train_df['comment_length'].mean():.1f} caractères")
print(f"  Médiane: {train_df['comment_length'].median():.1f} caractères")
print(f"  Min: {train_df['comment_length'].min()} caractères")
print(f"  Max: {train_df['comment_length'].max()} caractères")

# Histogramme des longueurs
plt.figure(figsize=(10, 6))
plt.hist(train_df['comment_length'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Distribution de la longueur des commentaires')
plt.xlabel('Longueur (caractères)')
plt.ylabel('Fréquence')
plt.grid(True, alpha=0.3)
plt.show()

## 6. Analyse RGPD - Risques Identifiés

In [ ]:
print("⚖️ ANALYSE RGPD - RISQUES IDENTIFIÉS:")
print("=" * 50)

risks = []

# Vérifier les colonnes de données personnelles
personal_columns = ['username', 'email', 'ip_address', 'phone_number', 'country']
found_personal_columns = [col for col in personal_columns if col in train_df.columns]

if found_personal_columns:
    risks.append(f"🔴 ÉLEVÉ: Colonnes de données personnelles présentes: {found_personal_columns}")

# Vérifier les entités dans les commentaires
risky_entities = ['PERSON', 'ORG', 'GPE']
found_risky_entities = [ent for ent in risky_entities if ent in entity_counts]

if found_risky_entities:
    risks.append(f"🟡 MOYEN: Entités nommées dans les commentaires: {found_risky_entities}")

# Vérifier les patterns de données personnelles
risky_patterns = [pattern for pattern, info in personal_data_found.items() if info['count'] > 0]

if risky_patterns:
    risks.append(f"🟡 MOYEN: Patterns de données personnelles: {risky_patterns}")

if not risks:
    risks.append("🟢 FAIBLE: Aucun risque majeur identifié")

for risk in risks:
    print(f"  {risk}")

print("\n📋 ACTIONS REQUISES:")
print("  1. ✅ Anonymiser toutes les colonnes de données personnelles")
print("  2. ✅ Appliquer NER sur les commentaires pour masquer les entités")
print("  3. ✅ Utiliser des patterns regex pour nettoyer les données restantes")
print("  4. ✅ Sauvegarder les versions anonymisées dans data/anonymized/")
print("  5. ✅ Compléter le registre RGPD")

## 7. Conclusion et Prochaines Étapes

In [ ]:
print("✅ EXPLORATION TERMINÉE - RÉSUMÉ:")
print("=" * 40)

print(f"\n📊 Données analysées:")
print(f"  - {train_df.shape[0]:,} commentaires d'entraînement")
print(f"  - {test_df.shape[0]:,} commentaires de test")
print(f"  - {len(found_personal_columns)} types de données personnelles ajoutées")
print(f"  - {len(entities)} entités détectées par spaCy")

print(f"\n🎯 Prochaines étapes:")
print(f"  1. Créer le script d'anonymisation (scripts/anonymize.py)")
print(f"  2. Traiter les données avec spaCy NER")
print(f"  3. Appliquer les patterns regex")
print(f"  4. Sauvegarder les versions anonymisées")
print(f"  5. Créer le rapport de comparaison avant/après")

print(f"\n🛡️ Priorité RGPD: ÉLEVÉE")
print(f"  ⚠️  Anonymisation obligatoire avant utilisation du modèle IA")